# Atividade 02: Atributos Categóricos e Valores Faltantes
### Aluno: Alisson da Silva Vieira

# Bibliotecas utilizadas
- Numpy: É uma biblioteca fundamental para computação científica em Python, que fornece um objeto de matriz multidimensional, vários objetos derivados (como matrizes e matrizes mascaradas) e uma variedade de rotinas para operações rápidas em matrizes.
- Pandas: É uma biblioteca que fornece estruturas de dados rápidas, flexíveis e expressivas projetadas para tornar o trabalho com dados "relacionais" ou "rotulados" fácil e intuitivo. Tem como objetivo ser o bloco de construção fundamental de alto nível para fazer análises de dados.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scipy.stats import ttest_ind_from_stats
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

%matplotlib inline

Inicialmente abrimos o arquivo .csv, e conseguimos analisar os primeiros 5 dados do arquivo.

In [ ]:
df = pd.read_csv('data/agaricus_lepiota_small_c.csv')
df.head()

Conseguimos analisar também as colunas. </br>
De cara já verificamos que apenas a coluna 'stalk-root' possui valores faltantes.

In [ ]:
df.info()

E para uma melhor análise, conseguimos fazer uma breve análise estatística, agrupando nossos dados pela coluna 'class'.

In [ ]:
df.groupby('class').describe()

Como os dados da coluna 'stalk-root' possuem valores faltantes, irei tirar essa coluna do dataset. <br>
Minha hipotese é de que como temos várias outras colunas, tirar a coluna 'stalk-root' pode não prejudir a análise.

In [ ]:
df = df.drop(['stalk-root'], axis=1)

Conseguimos perceber que todos os nossos dados possuem valores ordinais, então temos que realizar uma transformação para que os dados sejam categóricos.

In [ ]:
transformers = [
    ('v_class', OrdinalEncoder(categories=[['e', 'p']]), ['class']),
    ('v_cap-shape', OrdinalEncoder(), ['cap-shape']),
    ('v_cap-surface', OrdinalEncoder(), ['cap-surface']),
    ('v_cap-color', OrdinalEncoder(), ['cap-color']),
    ('v_bruises', OrdinalEncoder(categories=[['t', 'f']]), ['bruises']),
    ('v_odor', OrdinalEncoder(), ['odor']),
    ('v_gill-attachment', OrdinalEncoder(), ['gill-attachment']),
    ('v_gill-spacing', OrdinalEncoder(categories=[['c', 'w', 'd']]), ['gill-spacing']),
    ('v_gill-size', OrdinalEncoder(categories=[['b', 'n']]), ['gill-size']),
    ('v_gill-color', OrdinalEncoder(), ['gill-color']),
    ('v_stalk-shape', OrdinalEncoder(categories=[['e', 't']]), ['stalk-shape']),
    # ('v_stalk-root', OrdinalEncoder(), ['stalk-root']),
    ('v_stalk-surface-above-ring', OrdinalEncoder(), ['stalk-surface-above-ring']),
    ('v_stalk-surface-below-ring', OrdinalEncoder(), ['stalk-surface-below-ring']),
    ('v_stalk-color-above-ring', OrdinalEncoder(), ['stalk-color-above-ring']),
    ('v_stalk-color-below-ring', OrdinalEncoder(), ['stalk-color-below-ring']),
    ('v_veil-type', OrdinalEncoder(categories=[['p', 'u']]), ['veil-type']),
    ('v_veil-color', OrdinalEncoder(), ['veil-color']),
    ('v_ring-number', OrdinalEncoder(categories=[['n', 'o', 't']]), ['ring-number']),
    ('v_ring-type', OrdinalEncoder(), ['ring-type']),
    ('v_spore-print-color', OrdinalEncoder(), ['spore-print-color']),
    ('v_population', OrdinalEncoder(), ['population']),
    ('v_habitat', OrdinalEncoder(), ['habitat'])
]

ct = ColumnTransformer(transformers=transformers)

X_oe = ct.fit_transform(df)
df_oe = pd.DataFrame(X_oe, columns=df.columns)

Esse é o resultado depois da transfomração dos dados.

In [ ]:
df_oe.head()

Cosneguimos observar melhor o tipo dos valores das colunas.

In [ ]:
X_oe = df_oe.values
y_oe = df_oe['class'].values

df_oe.info()

Abaixo se encontra as funções responsáveis pela classificação dos dados, usando o SVM e o KNN

In [ ]:
'''
    ### Funções referente ao classificador KNN ####
'''

def validacaoCruzadaKnn(X, y, kVias = 10):

    print('Iniciando a validação cruzada com o classificado KNN...')

    # acuracias
    acuracias = []

    # usar o protocolo de validação cruzada estratificada
    skf = StratifiedKFold(n_splits=kVias, shuffle=True, random_state=1)

    pgb = tqdm(total=kVias, desc="Fold's avaliados")

    for idx_treino, idx_teste in skf.split(X, y):

        # extrair as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        X_treino = X[idx_treino]
        y_treino = y[idx_treino]
        
        # extrair as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        X_teste = X[idx_teste]
        y_teste = y[idx_teste]

        # separar as instâncias de treinamento entre treinamento e validação para a otimização do hiperparâmetro k
        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2, stratify=y_treino, shuffle=True, random_state=1)

        params = {'n_neighbors' : range(1,30,2)}

        # ss = StandardScaler()
        # ss.fit(X_treino)
        # X_treino = ss.transform(X_treino)
        # X_teste = ss.transform(X_teste)
        # X_val = ss.transform(X_val)

        knn = GridSearchCV(KNeighborsClassifier(), params, cv=StratifiedKFold(n_splits=5))
        knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
        
        pred = knn.predict(X_teste)

        print('f1-socre: ', f1_score(y_teste, pred), '\nOutro: ', accuracy_score(y_teste, pred))

        # calcular a acurácia no conjunto de testes desta iteração e salvar na lista.
        acuracias.append(accuracy_score(y_teste, pred))

        pgb.update(1)
        
    pgb.close()
    
    return acuracias

'''
    ### Funções referente ao classificador SVM ####
'''

def validacaoCruzadaSvm(X, y, cv_splits, Cs=[1], gammas=['scale']):

    print('Iniciando a validação cruzada com o classificador SVM...')

    parameters = [{
        'gamma': gammas,
        'C': Cs,
        'kernel': ['rbf']
    }]

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []
    
    pgb = tqdm(total=cv_splits, desc="Fold's avaliados")
    
    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)

        # ss = StandardScaler()
        # ss.fit(X_treino)
        # X_treino = ss.transform(X_treino)
        # X_teste = ss.transform(X_teste)
        # X_val = ss.transform(X_val)

        svm = GridSearchCV(SVC(), parameters, cv=StratifiedKFold(n_splits=5))
        svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
        
        pred = svm.predict(X_teste)

        print('f1-socre: ', f1_score(y_teste, pred), '\nOutro: ', accuracy_score(y_teste, pred))

        acuracias.append(accuracy_score(y_teste, pred))

        pgb.update(1)
        
    pgb.close()
    
    return acuracias

'''
    ### Funções auxiliares ####
'''

def showResult(acc, legend):
    print('Resultado ', legend, '\n    >> Acc mínima: ', round(min(acc), 3), '%\n    >> Acc máxima: ', round(max(acc), 3), '%')
    print('    >> Média: ', round(np.mean(acc), 3), '\n    >> Desvio padrão: ', round(np.std(acc), 3), '\n')

    return np.mean(acc), np.std(acc)

def hipoteseNula(media1, std1, values1, media2, std2, values2, alpha=0.05):
    pvalor = ttest_ind_from_stats(media1, std1, len(values1), media2, std2, len(values2))[1]
    
    return pvalor <= alpha

In [ ]:
accKnn = validacaoCruzadaKnn(X_oe, y_oe)
accSvm = validacaoCruzadaSvm(X_oe, y_oe, 10, Cs=[1, 10, 100, 1000], gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4])

In [ ]:
mediaKnn, stdKnn = showResult(accKnn, 'knn')
mediaSvm, stdSvm = showResult(accSvm, 'svm')

hpnula = hipoteseNula(mediaKnn, stdKnn, accKnn, mediaSvm, stdSvm, accSvm)

if hpnula:
    if mediaKnn > mediaSvm:
        classif = ' knn '
    else:
        classif = ' svm '

    text = 'verdadeira, podemos dizer que o' + classif + 'obteve um resultado melhor.'
else:
    text = 'falsa, não conseguimos afirmar que os dois resultados são estatisticamente diferentes.'

print('Hipotese nula:', text)